# Generatiivsed võrgud

Korduvad närvivõrgud (RNN-id) ja nende väratiga rakutüübid, nagu Long Short Term Memory Cells (LSTM-id) ja Gated Recurrent Units (GRU-d), pakkusid mehhanismi keele modelleerimiseks, st nad suudavad õppida sõnade järjestust ja anda ennustusi järgmise sõna kohta järjestuses. See võimaldab meil kasutada RNN-e **generatiivseteks ülesanneteks**, nagu tavaline tekstigeneratsioon, masintõlge ja isegi pildiallkirjade loomine.

RNN arhitektuuris, mida arutasime eelmises osas, genereeris iga RNN üksus järgmise varjatud oleku väljundina. Kuid me saame lisada igale korduva üksusele veel ühe väljundi, mis võimaldab meil luua **järjestuse** (mis on sama pikk kui algne järjestus). Lisaks saame kasutada RNN üksusi, mis ei võta igal sammul sisendit, vaid kasutavad ainult algoleku vektorit ja seejärel genereerivad väljundite järjestuse.

Selles märkmikus keskendume lihtsatele generatiivsetele mudelitele, mis aitavad meil teksti genereerida. Lihtsuse huvides loome **tähemärgi tasemel võrgu**, mis genereerib teksti täht-tähelt. Treeningu ajal peame võtma mõne tekstikorpuse ja jagama selle tähemärkide järjestusteks.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Tähemärkide sõnavara loomine

Tähemärkide tasemel generatiivse võrgu loomiseks peame teksti jagama üksikuteks tähemärkideks, mitte sõnadeks. `TextVectorization` kiht, mida oleme varem kasutanud, ei suuda seda teha, seega on meil kaks võimalust:

* Laadida tekst käsitsi ja teha tokeniseerimine "käsitsi", nagu [selles ametlikus Keras näites](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Kasutada `Tokenizer` klassi tähemärkide tasemel tokeniseerimiseks.

Me valime teise variandi. `Tokenizer`-it saab kasutada ka sõnade tokeniseerimiseks, seega peaks olema lihtne vahetada tähemärkide tasemelt sõnade tasemele.

Tähemärkide tasemel tokeniseerimiseks peame lisama parameetri `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Me tahame kasutada ka ühte spetsiaalset tokenit, et tähistada **järjestuse lõppu**, mida me nimetame `<eos>`. Lisame selle käsitsi sõnavarasse:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Nüüd, et kodeerida teksti numbrijadadeks, saame kasutada:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Generatiivse RNN-i treenimine pealkirjade genereerimiseks

RNN-i treenimine uudiste pealkirjade genereerimiseks toimub järgmiselt. Igal sammul võtame ühe pealkirja, mis sisestatakse RNN-i, ja iga sisendmärgi puhul palume võrgul genereerida järgmine väljundmärk:

![Pilt, mis näitab RNN-i näidet sõna 'HELLO' genereerimisel.](../../../../../translated_images/et/rnn-generate.56c54afb52f9781d.webp)

Meie järjestuse viimase märgi puhul palume võrgul genereerida `<eos>` tokeni.

Peamine erinevus generatiivse RNN-i puhul, mida siin kasutame, seisneb selles, et võtame väljundi igast RNN-i sammust, mitte ainult viimasest rakust. Seda saab saavutada, määrates RNN-i rakule parameetri `return_sequences`.

Seega, treeningu ajal on võrgu sisendiks teatud pikkusega kodeeritud märkide järjestus ja väljundiks sama pikkusega järjestus, mis on nihutatud ühe elemendi võrra ja lõpetatud `<eos>`-iga. Minipartii koosneb mitmest sellisest järjestusest, ja me peame kasutama **täitmist**, et kõik järjestused ühtlustada.

Loome funktsioonid, mis muudavad meie jaoks andmekogu. Kuna soovime järjestusi täita minipartii tasemel, rühmitame esmalt andmekogu, kutsudes `.batch()`, ja seejärel rakendame `map`, et teha transformatsioon. Seega võtab transformatsioonifunktsioon parameetrina terve minipartii:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Mõned olulised asjad, mida me siin teeme:  
* Kõigepealt eraldame tegeliku teksti stringi tensorist  
* `text_to_sequences` teisendab stringide loendi täisarvude tensorite loendiks  
* `pad_sequences` täidab need tensorid nende maksimaalse pikkuseni  
* Lõpuks kodeerime kõik tähemärgid ühekordselt (one-hot encoding), teeme nihutamise ja lisame `<eos>`. Varsti näeme, miks me vajame ühekordselt kodeeritud tähemärke  

Siiski on see funktsioon **Pythonile omane**, st seda ei saa automaatselt tõlkida Tensorflow arvutusgraafikuks. Kui proovime seda funktsiooni otse kasutada `Dataset.map` funktsioonis, saame veateateid. Peame selle Pythonile omase kõne ümbritsema, kasutades `py_function` ümbrist:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Märkus**: Pythonic ja Tensorflow teisendusfunktsioonide eristamine võib tunduda veidi keeruline ning võite küsida, miks me ei teisenda andmekogumit tavaliste Pythoni funktsioonide abil enne, kui selle `fit`-i edastame. Kuigi see on kindlasti võimalik, on `Dataset.map` kasutamisel suur eelis, kuna andmete teisendamise torujuhe täidetakse Tensorflow arvutusgraafiku abil, mis kasutab ära GPU arvutusvõimsust ja vähendab vajadust andmeid CPU ja GPU vahel edasi-tagasi liigutada.

Nüüd saame luua oma generaatori võrgu ja alustada treenimist. See võib põhineda mis tahes korduvrakul, mida arutasime eelmises osas (lihtne, LSTM või GRU). Meie näites kasutame LSTM-i.

Kuna võrk võtab sisendiks tähemärke ja sõnavara suurus on üsna väike, ei ole meil vaja sisendkihtide jaoks embedding-kihte; üheselt kodeeritud sisend võib otse minna LSTM-rakku. Väljundkiht oleks `Dense` klassifikaator, mis teisendab LSTM-i väljundi üheselt kodeeritud tokenite numbriteks.

Lisaks, kuna töötame muutuva pikkusega jadadega, saame kasutada `Masking` kihti, et luua mask, mis ignoreerib täiendatud stringi osi. See ei ole rangelt vajalik, kuna meid ei huvita väga see, mis jääb `<eos>` tokenist kaugemale, kuid kasutame seda kihti, et saada kogemust selle tüüpi kihiga. `input_shape` oleks `(None, vocab_size)`, kus `None` tähistab muutuva pikkusega jada, ja väljundkuju on samuti `(None, vocab_size)`, nagu näete `summary`-st:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Väljundi genereerimine

Nüüd, kui oleme mudeli treeninud, tahame seda kasutada väljundi genereerimiseks. Kõigepealt vajame viisi, kuidas dekodeerida tekst, mis on esitatud tokenite numbrite jadana. Selleks võiksime kasutada funktsiooni `tokenizer.sequences_to_texts`; siiski ei tööta see hästi tähemärgi tasemel tokeniseerimisega. Seetõttu võtame tokenite sõnastiku tokeniseerijast (nimetatakse `word_index`), loome pöördkaardi ja kirjutame oma dekodeerimisfunktsiooni:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Nüüd alustame genereerimist. Võtame alguses stringi `start`, kodeerime selle järjestuseks `inp` ja igal sammul kutsume oma võrgu välja, et tuvastada järgmine tähemärk.

Võrgu väljund `out` on vektor, millel on `vocab_size` elementi, mis esindavad iga tokeni tõenäosust. Kõige tõenäolisema tokeni numbri saame leida, kasutades `argmax`. Seejärel lisame selle tähemärgi genereeritud tokenite loendisse ja jätkame genereerimist. Ühe tähemärgi genereerimise protsessi korratakse `size` korda, et luua vajalik arv tähemärke, ja lõpetame varakult, kui kohtame `eos_token`-i.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Väljundi proovivõtmine treeningu ajal

Kuna meil puuduvad kasulikud mõõdikud, nagu *täpsus*, on ainus viis näha, et meie mudel paraneb, **proovivõtmine** genereeritud stringist treeningu ajal. Selleks kasutame **tagasiside funktsioone** ehk funktsioone, mida saame edastada `fit` funktsioonile ja mis kutsutakse treeningu käigus perioodiliselt välja.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


See näide genereerib juba üsna head teksti, kuid seda saab mitmel viisil veelgi paremaks muuta:
* **Rohkem teksti**. Oleme oma ülesande jaoks kasutanud ainult pealkirju, kuid võite katsetada täistekstiga. Pidage meeles, et RNN-id ei ole eriti head pikkade järjestuste käsitlemisel, seega on mõistlik need kas jagada lühemateks lauseteks või alati treenida fikseeritud järjestuse pikkusega, mille väärtus on eelnevalt määratud `num_chars` (näiteks 256). Võite proovida ülaltoodud näidet muuta selliseks arhitektuuriks, kasutades inspiratsiooniks [ametlikku Keras õpetust](https://keras.io/examples/generative/lstm_character_level_text_generation/).
* **Mitmekihiline LSTM**. Tasub proovida 2 või 3 LSTM-rakkude kihti. Nagu mainisime eelmises osas, iga LSTM kiht eraldab tekstist teatud mustreid, ja tähemärgi tasemel generaatori puhul võime eeldada, et madalam LSTM tase vastutab silpide eraldamise eest, ning kõrgemad tasemed - sõnade ja sõnakombinatsioonide eest. Seda saab lihtsalt rakendada, andes LSTM konstruktorile kihtide arvu parameetri.
* Võite katsetada ka **GRU üksustega** ja vaadata, millised annavad paremaid tulemusi, ning **erinevate varjatud kihtide suurustega**. Liiga suur varjatud kiht võib viia üleõppimisele (näiteks õpib võrk täpse teksti), ja väiksem suurus ei pruugi anda häid tulemusi.


## Pehme teksti genereerimine ja temperatuur

Eelmises `generate` definitsioonis valisime alati järgmise tähemärgina selle, millel oli kõrgeim tõenäosus genereeritud tekstis. See tõi sageli kaasa olukorra, kus tekst "kordas" samu tähemärkide jadasid ikka ja jälle, nagu selles näites:
```
today of the second the company and a second the company ...
```

Kuid kui vaatame järgmise tähemärgi tõenäosusjaotust, võib juhtuda, et mõne kõrgeima tõenäosuse vahe ei ole suur, näiteks ühel tähemärgil võib olla tõenäosus 0,2 ja teisel 0,19 jne. Näiteks, kui otsime järgmist tähemärki jadas '*play*', võib järgmine tähemärk sama hästi olla kas tühik või **e** (nagu sõnas *player*).

See viib meid järelduseni, et ei ole alati "õiglane" valida tähemärki, millel on kõrgem tõenäosus, sest ka teise kõrgeima valimine võib viia tähendusliku tekstini. Mõistlikum on **valida juhuslikult** tähemärke võrgu väljundi antud tõenäosusjaotuse põhjal.

Seda juhuslikku valimist saab teha `np.multinomial` funktsiooni abil, mis rakendab nn **multinomiaalset jaotust**. Allpool on defineeritud funktsioon, mis rakendab seda **pehmet** teksti genereerimist:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Oleme lisanud veel ühe parameetri nimega **temperatuur**, mida kasutatakse näitamaks, kui tugevalt peaksime järgima kõrgeimat tõenäosust. Kui temperatuur on 1.0, teeme õiglast multinomiaalse valimit, ja kui temperatuur läheneb lõpmatusele - muutuvad kõik tõenäosused võrdseks ning me valime järgmise tähemärgi juhuslikult. Allolevas näites näeme, et tekst muutub mõttetuks, kui temperatuuri liiga palju suurendame, ja meenutab "tsüklilist" rangelt genereeritud teksti, kui see läheneb 0-le.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
